In [306]:
import pandas as pd
import plotly.express as px
import numpy as np
import os

segment = 'method' # 'contract' # 'project'

In [307]:
prepend = 'csv_exports/whale_actions/'

op_df = pd.read_csv(prepend + 'op_whale_actions_22_01_18.csv')
op_df['chain'] = 'Optimism'

l1_df = pd.read_csv(prepend + 'l1_whale_actions_22_01_18.csv')
l1_df['chain'] = 'Ethereum'

arb_df = pd.read_csv(prepend + 'arbi_whale_actions_22_01_18.csv')
arb_df['chain'] = 'Arbitrum'

df = pd.concat([op_df, l1_df, arb_df])
df = df[df['project'] != '0x0000000000000000000000000000000000000000']
df = df.fillna(0)

df.sample(20)



,address,token_direction,project,contract,method,usd_value,num_tfers,num_txs,chain
140534,0xf89d7b9c864f589bbf53a82105107622b35eaa40,out,0xc7a4b31154098a71ef7f355eff60abd5ba711445,0,transfer,1.631418e+04,6,6,Ethereum
111287,0x7819d643b93746a28f59c59fb0e9eb49cd3d839f,in,0x71db5b6b2b7ebb785eab3b55861824977a1e0323,0,0x,2.005940e+06,2,2,Ethereum
125838,0x8e04af7f7c76daa9ab429b1340e0327b5b835748,out,0x8c00dda00f9e2aabde88a4d17f9ef9fce265592f,0,transfer,8.304795e+05,15,15,Ethereum
201535,0x27996288e7022f04ce1e619fa20bd1202f444ab2,out,0x22648c12acd87912ea1710357b1302c6a4154ebc,0,anySwapOutUnderlying,1.034867e+05,1,1,Ethereum
179310,0x1a87e6a0273e1fc31b70212a05d48f36725c5196,in,0x48b39515dcdb1288e9e2b2d757697da139dd275f,0,transfer,9.051417e+05,2,2,Ethereum
105760,0xfe26806edbbe2ccb95a910fc79cbd41ca4bf9eb8,out,near,ERC20Locker,lockToken,7.532580e+04,2,2,Ethereum
248265,0x193db18a5ef9a0320b7374c1fe8af976235f3211,in,0x6131b5fae19ea4f9d964eac0408e4408b66337b5,0,0xe21fd0e9,3.374812e+05,10,10,Ethereum
145616,0xe63d56545cf81bfb24a97f98dd29032cfb5b34dd,in,OTHER,OTHER,OTHER,2.202798e-01,5,5,Ethereum
182173,0x5b5a922e065a425ee7e9b70185ba563ef270afcd,in,0x7b6c63ecc3d0fa75dc4885f0dbbe4a4bffd45562,0,transfer,3.425114e+04,3,3,Ethereum
47502,0x9d137567c35d6704b9155a88c8480237c7b24705,in,0x21a31ee1afc51d94c2efccaa2092ad1028285549,0,transfer,3.531000e+05,1,1,Ethereum


In [308]:

df['usd_value_in'] = np.where(df['token_direction'] == 'in',df['usd_value'], 0)
df['usd_value_out'] = np.where(df['token_direction'] == 'out',df['usd_value'], 0)
df['usd_value_net'] = df['usd_value_in'] - df['usd_value_out']
df['usd_value_net'] = np.where(df['usd_value_net'] < 0, 0, df['usd_value_net'])
val_cols = ['usd_value','usd_value_in','usd_value_out','usd_value_net']

In [309]:
df_addr = df.groupby(['chain','address','project','contract','method']).sum()
df_addr.reset_index(inplace=True)

for val in val_cols:
        df_addr[val + '_pct_share'] = df_addr.groupby(['address','chain'])[val].transform(lambda x: x / x.sum())
# display(df_addr)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/4210915753.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_addr = df.groupby(['chain','address','project','contract','method']).sum()


In [310]:
group_list = ['chain','project','contract','method']
dlist = ['chain','project','contract','method','action'\
                ,'usd_value','usd_value_in','usd_value_out','usd_value_net'\
                ,'usd_value_pct_share','usd_value_in_pct_share','usd_value_out_pct_share','usd_value_net_pct_share'
                ,'address' \
                ]

if segment == 'project':
        group_list.drop('method',inplace=True)
        group_list.drop('contract',inplace=True)
        dlist.drop('method',inplace=True)
        dlist.drop('contract',inplace=True)
elif segment == 'contract':
        group_list.drop('method',inplace=True)
        dlist.drop('method',inplace=True)

In [311]:
df_dollar = df[['chain','project','contract','method','usd_value','usd_value_in','usd_value_out','usd_value_net','address']]
df_dollar = df_dollar.groupby(['chain','project','contract','method']).agg({
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
        ,'address':'nunique'
})
df_dollar.reset_index(inplace=True)
df_dollar.name = 'df_dollar'
for val in val_cols:
        df_dollar[val + '_pct_share'] =  df_dollar[val] / ( df_dollar[val].sum() )


In [312]:
# Get the average volume share across all whales
# i.e. once someone is determined to be a whale, tkae their pct transfer share, and then we avg across rather than $ weight.
# The reason to not dollar-weight is that we care more about genetating the average individual user experience, rather than the whaliest whale.

df_action = df_addr.groupby(['chain','project','contract','method']).agg({
        'usd_value_pct_share':'mean'
        ,'usd_value_in_pct_share':'mean'
        ,'usd_value_out_pct_share':'mean'
        ,'usd_value_net_pct_share':'mean'
        ,'usd_value':'mean'
        ,'usd_value_in':'mean'
        ,'usd_value_out':'mean'
        ,'usd_value_net':'mean'
        ,'address':'nunique'
})

df_action_by_app = df_addr.groupby(['chain','project','address']).agg({
        # 'usd_value_pct_share':'mean'
        # ,'usd_value_in_pct_share':'mean'
        # ,'usd_value_out_pct_share':'mean'
        # ,'usd_value_net_pct_share':'mean'
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
})

df_action_by_app.reset_index(inplace=True)

for val in val_cols:
        df_action_by_app[val + '_pct_share'] = df_action_by_app.groupby(['chain','address'])[val].transform(lambda x: x / x.sum())

df_action_by_app = df_action_by_app.groupby(['chain','project']).agg({
        'usd_value_pct_share':'mean'
        ,'usd_value_in_pct_share':'mean'
        ,'usd_value_out_pct_share':'mean'
        ,'usd_value_net_pct_share':'mean'
        # 'usd_value':'mean'
        # ,'usd_value_in':'mean'
        # ,'usd_value_out':'mean'
        # ,'usd_value_net':'mean'
        # ,'address':'nunique'
})


df_action.reset_index(inplace=True)
df_action_by_app.reset_index(inplace=True)

df_action.name = 'df_action'
df_action_by_app.name = 'df_action_by_app'

display(df_action)

,chain,project,contract,method,usd_value_pct_share,usd_value_in_pct_share,usd_value_out_pct_share,usd_value_net_pct_share,usd_value,usd_value_in,usd_value_out,usd_value_net,address
0,Arbitrum,0x0000000000be6d8381e3a01c19e3b3c2b6d9c7cd,0,0xed5b0df3,0.232680,0.063109,0.365967,0.063109,5.829281e+06,695814.462200,5.133467e+06,695814.462200,1
1,Arbitrum,0x000000d4728ded3cc178ffa58355e6b57b931ee2,0,0xed5b0df3,0.367702,0.028867,0.634033,0.028867,9.211949e+06,318280.444237,8.893669e+06,318280.444237,1
2,Arbitrum,0x000c3ff63086a0a34b3a4750a66adf206753fc6b,0,transfer,0.000233,0.000456,0.000000,0.000456,1.007082e+04,10070.820000,0.000000e+00,10070.820000,1
3,Arbitrum,0x00680642adb56ea659d96a91e81099c43886f513,0,transfer,0.575438,0.701914,0.297549,0.701914,4.997175e+05,418897.352500,8.082018e+04,418897.352500,1
4,Arbitrum,0x008ba48b1c005edbc76d75c7143c868576f1fbbf,0,transfer,0.000262,0.000000,0.000536,0.000000,1.129990e+04,0.000000,1.129990e+04,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139229,Optimism,Wepiggy,PEther,borrow,0.001958,0.004165,0.000000,0.004165,2.318055e+05,231805.500000,0.000000e+00,231805.500000,1
139230,Optimism,Wepiggy,PEther,repayBorrow,0.000144,0.000000,0.000271,0.000000,1.699907e+04,0.000000,1.699907e+04,0.000000,1
139231,Optimism,Woofi,0,withdraw,0.030797,0.060700,0.000000,0.060700,1.004902e+05,100490.201009,0.000000e+00,100490.201009,1
139232,Optimism,Woofi,WooSuperChargerVault,deposit,0.030780,0.000000,0.062481,0.000000,1.004344e+05,0.000000,1.004344e+05,0.000000,1


In [313]:
dfs = [df_action, df_dollar]


if not os.path.exists("csv_outputs"):
        os.mkdir("csv_outputs")
        
for i, d in enumerate(dfs):
        original_name = d.name
        d.reset_index(inplace=True)
        if segment == 'method':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str) + ' | ' + d['method'].astype(str)
        elif segment == 'contract':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str)
        elif segment == 'project':
                d['action'] = d['project'].astype(str)
        
        # d = d[dlist]
        
        dfs[i] = d
        d.name = original_name
        
#dumb hardcode, tried to do it fancy, but whatever
df_action = dfs[0]
df_dollar = dfs[1]

df_action.to_csv('csv_outputs/whale_actions_share.csv')
df_dollar.to_csv('csv_outputs/whale_actions_share_dollar_weight.csv')

df_action.sample(20)
# gix = px.pie(df_action, values='usd_value_out_pct_share', names='action', title='Share of Actions')

,index,chain,project,contract,method,usd_value_pct_share,usd_value_in_pct_share,usd_value_out_pct_share,usd_value_net_pct_share,usd_value,usd_value_in,usd_value_out,usd_value_net,address,action
34687,34687,Ethereum,0x3ce6de6dc2bb308e838556821e9a900cec36e2be,0,transfer,0.249830,0.000000,0.499660,0.000000,5.533603e+05,0.000000,5.533603e+05,0.000000,1,0x3ce6de6dc2bb308e838556821e9a900cec36e2be - 0...
7934,7934,Ethereum,0x098b88f9d9e40fcfa5e49c40293d7e7725cf6a11,0,transfer,0.000015,0.000000,0.000028,0.000000,3.549045e+04,0.000000,3.549045e+04,0.000000,1,0x098b88f9d9e40fcfa5e49c40293d7e7725cf6a11 - 0...
6158,6158,Ethereum,0x063c89bf4a5d6716845614861d9823c70fcc8126,0,transfer,0.027118,0.000000,0.054240,0.000000,4.519548e+04,0.000000,4.519548e+04,0.000000,1,0x063c89bf4a5d6716845614861d9823c70fcc8126 - 0...
130234,130234,Ethereum,0xf376acb838a3f1cf6f00217c5391caade14d9168,0,transfer,0.079663,0.000000,0.161793,0.000000,6.546033e+04,0.000000,6.546033e+04,0.000000,1,0xf376acb838a3f1cf6f00217c5391caade14d9168 - 0...
110965,110965,Ethereum,0xce66c6a88bd7bec215aa04fda4cf7c81055521d0,0,transfer,0.000300,0.000940,0.000000,0.000940,2.812163e+04,28121.632000,0.000000e+00,28121.632000,1,0xce66c6a88bd7bec215aa04fda4cf7c81055521d0 - 0...
67086,67086,Ethereum,0x7ab7728cc6ada51f468e27007768f29d265cfbb2,0,transfer,0.040984,0.077467,0.004684,0.077467,1.886495e+05,168915.998732,1.973347e+04,168915.998732,2,0x7ab7728cc6ada51f468e27007768f29d265cfbb2 - 0...
127900,127900,Ethereum,0xef08be9bc892eea8bff9a856c8f84b77b0e9c9c8,0,transfer,0.017865,0.000000,0.035872,0.000000,4.720697e+04,0.000000,4.720697e+04,0.000000,1,0xef08be9bc892eea8bff9a856c8f84b77b0e9c9c8 - 0...
8183,8183,Ethereum,0x0a08c10e2779c06a76e1ac468613b795b2783612,0,transfer,0.267871,0.000000,0.733333,0.000000,5.538951e+05,0.000000,5.538951e+05,0.000000,1,0x0a08c10e2779c06a76e1ac468613b795b2783612 - 0...
46255,46255,Ethereum,0x52f020b18b35d86c1ab0a48a121147010eeb4475,0,transfer,0.001476,0.000000,0.002965,0.000000,2.014607e+04,0.000000,2.014607e+04,0.000000,1,0x52f020b18b35d86c1ab0a48a121147010eeb4475 - 0...
91563,91563,Ethereum,0xa999c7a999ee179a88ef7efccbe9dd010923bc0e,0,fillTakerSignedOtcOrder,0.000411,0.000408,0.000415,0.000408,2.099207e+04,10454.313344,1.053776e+04,10454.313344,1,0xa999c7a999ee179a88ef7efccbe9dd010923bc0e - 0...


In [314]:
print('Dollar-Weighted')
chains = df_dollar['chain'].drop_duplicates().to_list()
# print('Top Dollar Weighted Actions by Chain')

for c in chains:
        tdf = df_dollar[df_dollar['chain'] == c]
        sumdf = []
        print('Sum by Action')
        sumdf = tdf.groupby(['chain','project','action']).sum()
        sumdf.reset_index(inplace=True)
        
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','action','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )
        print('Sum by App')
        sumdf = tdf.groupby(['chain','project']).sum()
        sumdf.reset_index(inplace=True)
        for val in val_cols:
                sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())

        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )

        print(sumdf[['usd_value_out_pct_share_100']].sum())

Dollar-Weighted
Sum by Action


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1850225724.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).sum()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
2562,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInputSingle,9.523727,9.255335
2561,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInput,5.702290,4.862586
1927,Arbitrum,celer_network,celer_network - Bridge | addLiquidity,5.504098,0.000000
2563,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactOutput,4.944629,5.567669
2015,Arbitrum,gmx,gmx - PositionRouter | 0xf2ae372f,3.998533,0.000000
536,Arbitrum,0x521709b3cd7f07e29722be0ba28a8ce0e806dbc3,0x521709b3cd7f07e29722be0ba28a8ce0e806dbc3 - 0...,3.286356,1.529717
343,Arbitrum,0x3405a1bd46b85c5c029483fbecf2f3e611026e45,0x3405a1bd46b85c5c029483fbecf2f3e611026e45 - 0...,3.059880,0.000000
2568,Arbitrum,uniswap_v3,uniswap_v3 - Pair | multicall,2.852370,3.406012
2020,Arbitrum,gmx,gmx - Vault | 0x364e2311,2.278467,0.000000
1891,Arbitrum,OTHER,OTHER - OTHER | OTHER,2.271972,2.345757


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1850225724.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


...

,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
663,Optimism,Celer,Celer - Bridge | addLiquidity,17.775668,0.000000
944,Optimism,Uniswap,Uniswap - Pool | 0x608d60eb,15.668466,15.976778
761,Optimism,Multichain,Multichain - 0 | anySwapOutUnderlying,5.628581,0.000000
950,Optimism,Uniswap,Uniswap - Pool | exactInputSingle,3.548631,3.762533
736,Optimism,Kyberswap,Kyberswap - AntiSnipAttackPositionManager | mu...,2.776817,4.930665
633,Optimism,0xff3cdca6db902617e6ae4133cb45df41c38222b1,0xff3cdca6db902617e6ae4133cb45df41c38222b1 - 0...,2.594831,0.006871
375,Optimism,0xb1feccccaad6429899a9401dcb19f58a19d1afe6,0xb1feccccaad6429899a9401dcb19f58a19d1afe6 - 0...,2.323820,0.000000
666,Optimism,Celer,Celer - Bridge | send,1.833812,0.000000
741,Optimism,Kyberswap,Kyberswap - Elastic_Pool | multicall,1.822989,0.000000
804,Optimism,Socket,Socket - 0 | outboundTransferTo,1.733445,0.000000


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1850225724.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


,chain,project,usd_value_out_pct_share_100,usd_value_net_pct_share_100
576,Optimism,Uniswap,23.300819,25.730368
533,Optimism,Celer,20.354813,24.457077
553,Optimism,Multichain,6.270284,6.020605
549,Optimism,Kyberswap,5.180304,5.265309
524,Optimism,0xff3cdca6db902617e6ae4133cb45df41c38222b1,2.639540,0.006871
526,Optimism,Aave,2.598473,4.172976
577,Optimism,Uniswap V3,2.470609,3.366635
571,Optimism,Sonne Finance,2.432443,2.811540
357,Optimism,0xb1feccccaad6429899a9401dcb19f58a19d1afe6,2.323820,0.000000
537,Optimism,Dforce,1.920007,1.210225


usd_value_out_pct_share_100    100.0
dtype: float64


In [315]:
print('Address-Weighted')
# print(df_action.columns)
for c in chains:
        tdf = df_action[df_action['chain'] == c]
        # display(tdf)
        tdf = tdf[tdf['address'] >=20]
        sumdf = tdf
        print('Sum by Action')
        # sumdf = tdf.groupby(['chain','project','action']).sum()
        # for val in val_cols:
        #         sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        sumdf = tdf.groupby(['chain','project','action']).mean()
        sumdf.reset_index(inplace=True)
        # print(sumdf.columns)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','action','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )
        print('Sum by App')
        sumdf = df_action_by_app.copy()
        # for val in val_cols:
        #         sumdf[val + '_pct_share'] = sumdf.groupby(['chain'])[val].transform(lambda x: x / x.sum())
        sumdf = tdf.groupby(['chain','project']).mean()
        sumdf.reset_index(inplace=True)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','usd_value_out_pct_share_100','usd_value_net_pct_share_100','address']]
        display( sumdf.head(10) )

Address-Weighted
Sum by Action
Index(['chain', 'project', 'action', 'index', 'usd_value_pct_share',
       'usd_value_in_pct_share', 'usd_value_out_pct_share',
       'usd_value_net_pct_share', 'usd_value', 'usd_value_in', 'usd_value_out',
       'usd_value_net', 'address'],
      dtype='object')


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/374270540.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).mean()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
17,Arbitrum,gmx,gmx - PositionRouter | 0xf2ae372f,52.366119,0.000000
19,Arbitrum,gmx,gmx - Vault | 0x364e2311,51.861224,0.000000
9,Arbitrum,0xf89d7b9c864f589bbf53a82105107622b35eaa40,0xf89d7b9c864f589bbf53a82105107622b35eaa40 - 0...,46.153846,17.144626
13,Arbitrum,across_v2,across_v2 - Arbitrum_SpokePool | deposit,42.060198,0.000000
25,Arbitrum,sushi,sushi - Pair | swapExactTokensForETH,38.122838,0.000000
7,Arbitrum,0x892785f33cdee22a30aef750f285e18c18040c3e,0x892785f33cdee22a30aef750f285e18c18040c3e - 0...,35.303184,0.000000
8,Arbitrum,0xb38e8c17e38363af6ebdcb3dae12e0243582891d,0xb38e8c17e38363af6ebdcb3dae12e0243582891d - 0...,34.375000,18.606047
23,Arbitrum,gmx,gmx - sGMX | 0xf3daeacc,32.593963,0.000000
28,Arbitrum,uniswap_v3,uniswap_v3 - Pair | mint,28.874147,0.000000
3,Arbitrum,0x3405a1bd46b85c5c029483fbecf2f3e611026e45,0x3405a1bd46b85c5c029483fbecf2f3e611026e45 - 0...,28.503866,0.000000


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/374270540.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).mean()


...

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/374270540.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).mean()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
10,Optimism,Multichain,Multichain - 0 | anySwapOutUnderlying,33.031674,0.000000
14,Optimism,Velodrome,Velodrome - Pair | addLiquidity,30.905623,0.000000
8,Optimism,Celer,Celer - Bridge | send,25.135731,0.000000
1,Optimism,0xff3cdca6db902617e6ae4133cb45df41c38222b1,0xff3cdca6db902617e6ae4133cb45df41c38222b1 - 0...,21.100266,0.117743
4,Optimism,Aave,Aave - AToken | repay,20.145970,0.000000
5,Optimism,Aave,Aave - AToken | supply,13.831759,0.000000
13,Optimism,Uniswap,Uniswap - Pool | multicall,12.931811,11.305158
11,Optimism,OTHER,OTHER - OTHER | OTHER,3.782918,10.768166
2,Optimism,1inch,1inch - AggregationRouterV5 | swap,2.878059,23.089112
0,Optimism,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,0xacd03d601e5bb1b275bb94076ff46ed9d753435a - 0...,0.000000,19.371645


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/374270540.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).mean()


,chain,project,usd_value_out_pct_share_100,usd_value_net_pct_share_100,address
9,Optimism,Velodrome,30.905623,0.000000,21.0
1,Optimism,0xff3cdca6db902617e6ae4133cb45df41c38222b1,21.100266,0.117743,59.0
5,Optimism,Multichain,16.515837,17.171564,50.0
8,Optimism,Uniswap,12.931811,11.305158,21.0
4,Optimism,Celer,12.567866,10.473531,28.0
3,Optimism,Aave,8.494432,12.073350,23.5
6,Optimism,OTHER,3.782918,10.768166,188.0
2,Optimism,1inch,2.878059,23.089112,57.0
0,Optimism,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,0.000000,19.371645,23.0
7,Optimism,Stargate Finance,0.000000,35.821886,21.0
